# RLP WMS vs JP2 (2022)

This notebook compares a 2022 RLP JP2 tile with the historical WMS layer `rp_dop20_rgb_2022` from `rp_hkdop20`. It computes sharpness metrics and renders a 5-panel visualization (JP2, WMS, overlay, abs diff, heatmap).


In [ ]:
from io import BytesIO
from pathlib import Path

import numpy as np
import rasterio
from rasterio.windows import Window
from rasterio.windows import bounds as window_bounds
import requests
import truststore

truststore.inject_into_ssl()


from PIL import Image

JP2_URL = "https://geobasis-rlp.de/data/dop20rgb/current/jp2/dop20rgb_32_368_5492_2_rp_2022.jp2"
WMS_BASE = "https://geo4.service24.rlp.de/wms/rp_hkdop20.fcgi"
WMS_LAYER = "rp_dop20_rgb_2022"
WIN_SIZE = 512
JP2_PATH = Path("/tmp/dop20rgb_32_368_5492_2_rp_2022.jp2")

In [ ]:
if not JP2_PATH.exists():
    resp = requests.get(JP2_URL, timeout=60)
    resp.raise_for_status()
    JP2_PATH.write_bytes(resp.content)

with rasterio.open(JP2_PATH) as src:
    width = src.width
    height = src.height
    col_off = max(0, width // 2 - WIN_SIZE // 2)
    row_off = max(0, height // 2 - WIN_SIZE // 2)
    window = Window(col_off, row_off, WIN_SIZE, WIN_SIZE)
    data = src.read(window=window)
    if data.shape[0] > 3:
        data = data[:3]
    jp2_rgb = np.moveaxis(data, 0, -1)
    minx, miny, maxx, maxy = window_bounds(window, src.transform)

wms_url = (
    f"{WMS_BASE}?service=WMS&version=1.3.0&request=GetMap"
    f"&layers={WMS_LAYER}&styles="
    f"&crs=EPSG:25832&bbox={minx},{miny},{maxx},{maxy}"
    f"&width={WIN_SIZE}&height={WIN_SIZE}&format=image/png"
)
wms_resp = requests.get(wms_url, timeout=30)
wms_resp.raise_for_status()
wms_rgb = np.array(Image.open(BytesIO(wms_resp.content)).convert("RGB"))

In [ ]:
def to_gray(rgb):
    return (0.2126 * rgb[..., 0] + 0.7152 * rgb[..., 1] + 0.0722 * rgb[..., 2]).astype(np.float32)


def laplacian(img):
    return (
        -4 * img
        + np.roll(img, 1, axis=0)
        + np.roll(img, -1, axis=0)
        + np.roll(img, 1, axis=1)
        + np.roll(img, -1, axis=1)
    )


def grad_energy(img):
    dx = img[:, 1:] - img[:, :-1]
    dy = img[1:, :] - img[:-1, :]
    return (dx * dx).mean(), (dy * dy).mean(), (dx * dx).mean() + (dy * dy).mean()


jp2_gray = to_gray(jp2_rgb)
wms_gray = to_gray(wms_rgb)

jp2_var_lap = float(laplacian(jp2_gray).var())
wms_var_lap = float(laplacian(wms_gray).var())

jp2_dx, jp2_dy, jp2_grad = grad_energy(jp2_gray)
wms_dx, wms_dy, wms_grad = grad_energy(wms_gray)

diff = np.abs(jp2_rgb.astype(np.int16) - wms_rgb.astype(np.int16))
mean_diff = float(diff.mean())
max_diff = int(diff.max())
exact_pct = float((diff == 0).all(axis=2).mean() * 100)

print("variance of Laplacian (higher=sharper)")
print(f"  JP2: {jp2_var_lap:.3f}")
print(f"  WMS: {wms_var_lap:.3f}")
print("gradient energy (higher=sharper)")
print(f"  JP2: {jp2_grad:.3f} (dx {jp2_dx:.3f}, dy {jp2_dy:.3f})")
print(f"  WMS: {wms_grad:.3f} (dx {wms_dx:.3f}, dy {wms_dy:.3f})")
print("diff stats")
print(f"  mean abs diff: {mean_diff:.2f}")
print(f"  max diff: {max_diff}")
print(f"  exact match pixels: {exact_pct:.2f}%")

In [ ]:
# Build 5-panel visualization
from PIL import ImageDraw

diff_mean = diff.mean(axis=2)
scale = 255.0 / 30.0

# Overlay (red intensity)
red = np.clip(diff_mean * scale, 0, 255).astype(np.uint8)
overlay = wms_rgb.copy()
overlay[..., 0] = np.maximum(overlay[..., 0], red)
mask = red > 0
overlay[..., 1] = np.where(mask, (overlay[..., 1] * 0.7).astype(np.uint8), overlay[..., 1])
overlay[..., 2] = np.where(mask, (overlay[..., 2] * 0.7).astype(np.uint8), overlay[..., 2])

# Absolute diff grayscale
abs_gray = np.clip(diff_mean * scale, 0, 255).astype(np.uint8)
abs_rgb = np.stack([abs_gray] * 3, axis=2)

# Heatmap
norm = np.clip(diff_mean / 30.0, 0, 1)
colors = np.array(
    [
        [0, 0, 0],
        [0, 0, 255],
        [0, 255, 255],
        [255, 255, 0],
        [255, 0, 0],
    ],
    dtype=np.float32,
)
positions = np.array([0.0, 0.25, 0.5, 0.75, 1.0], dtype=np.float32)
heat = np.zeros((WIN_SIZE, WIN_SIZE, 3), dtype=np.float32)

for i in range(len(colors) - 1):
    low = positions[i]
    high = positions[i + 1]
    mask = (norm >= low) & (norm <= high)
    if not mask.any():
        continue
    t = (norm[mask] - low) / (high - low)
    c0 = colors[i]
    c1 = colors[i + 1]
    heat[mask] = c0 + (c1 - c0) * t[:, None]

heat_rgb = heat.astype(np.uint8)

panels = [
    (jp2_rgb, "JP2"),
    (wms_rgb, "WMS 2022"),
    (overlay, "Overlay"),
    (abs_rgb, "Abs diff"),
    (heat_rgb, "Heatmap"),
]

labeled = []
for img, title in panels:
    pil = Image.fromarray(img.astype(np.uint8), "RGB")
    draw = ImageDraw.Draw(pil)
    draw.rectangle([0, 0, 120, 18], fill=(0, 0, 0))
    draw.text((4, 2), title, fill=(255, 255, 255))
    labeled.append(pil)

out = Image.new("RGB", (WIN_SIZE * len(labeled), WIN_SIZE))
for idx, img in enumerate(labeled):
    out.paste(img, (idx * WIN_SIZE, 0))

out